# <center>处理数值型数据</center>

介绍多种将原始数值型数据转换成机器学习算法所需**特征**的方法。

---
## <center>数据的特征</center>

### 4.1 特征的缩放
#### 问题描述
将一个数值型特征的值缩放(rescale)到两个特定的值之间。
#### 解决方案
使用 scikti-learn 的 MinMaxScale() 方法缩放来一个特征数组。

#### 计算公式
x` = (x-min) / (max-min)

In [1]:
import numpy as np
from sklearn import preprocessing

In [2]:
# 创建特征数组
features = np.array([[-500.5], [-100.1], [0], [100.1], [900.9]])
# 创建缩放器
# 缩放区间为(0, 1)，其中特征数组中的最小值为区间下界；最大值为区间上界。
minmax_scale = preprocessing.MinMaxScaler(feature_range=(0, 1))

# 缩放特征的值
scale_feature = minmax_scale.fit_transform(features)
scale_feature

array([[0.        ],
       [0.28571429],
       [0.35714286],
       [0.42857143],
       [1.        ]])

### 4.2 特征的标准化
#### 问题描述
对一个特征进行转换，使其平均值为0、标准差为1。
#### 解决方案
使用 scikit-learn 的 StandardScaler() 方法能够同时执行这两个转换。

In [3]:
import numpy as np
from sklearn import preprocessing

In [4]:
# 创建特征数组
features = np.array([[-1000.1], [-200.2], [500.5], [600.6], [9000.9]])
# 创建缩放器
scaler = preprocessing.StandardScaler()

# 转换特征
standardized = scaler.fit_transform(features)
standardized

array([[-0.76058269],
       [-0.54177196],
       [-0.35009716],
       [-0.32271504],
       [ 1.97516685]])

### 两种方法的对比
|方法|称呼|作用|说明|
|--|--|--|--|
|MinMaxScale|**min-max**缩放|用于将特征设置在统一范围内|常用语神经网络|
|StandardScaler|标准化缩放|用于将特征缩放为大致符合正态分布|**没有特殊原因**，推荐标准化缩放|
|RobustScaler|分位数缩放|取中位数或者四分位数进行缩放|当数据中存在较大的异常值(两端稠密，中间稀疏)|

### 4.3 归一化观察值
#### 问题描述
对观察值的每一个特征进行缩放，使其拥有一致的范数(总长度为1)。
#### 解决方案
使用 Normalizer() 方法，并指定 norm 参数。

In [5]:
import numpy as np
from sklearn.preprocessing import Normalizer

In [6]:
# 创建特征矩阵
features = np.array([[0.5, 0.5], [1.1, 3.4], [1.5, 20.2], [1.63, 34.4], [10.9, 3.3]])

# 创建归一化器，使用欧式范数(也称为L2范数)
normalizer = Normalizer(norm='l2')
# 转换特征矩阵
normalizer.transform(features)

array([[0.70710678, 0.70710678],
       [0.30782029, 0.95144452],
       [0.07405353, 0.99725427],
       [0.04733062, 0.99887928],
       [0.95709822, 0.28976368]])

### 4.4 生成多项式和交互特征
当特征和目标值(预测值）之间存在非线性关系时，就需要创建高阶(x²、x³)多项式特征，来进行拟合。

当一个特征需要依赖另一个特征才能对目标值造成影响的情况，这时两个特征对目标值的作用是相互依赖的。就需要生成一个交互特征(将两个特征相乘)，来计算目标值。

例如：预测咖啡是否是甜的。需要考虑两个特征：是否加了糖；是否进行了搅拌。单独看两者中的一个，是无法确定咖啡是甜的。

#### 问题描述
创建多项式特征和交互特征。
#### 解决方案
使用 scikit-learn 的 StandardScaler() 方法能够同时执行这两个转换。

In [7]:
import numpy as np
from sklearn.preprocessing import PolynomialFeatures

In [8]:
# 创建特征矩阵
features = np.array([[2, 3], [2, 3], [2, 3]])
# 创建PolynomialFeatures对象。degree参数决定了多项式的最高阶数；默认包含交互特征
polynomial_interation = PolynomialFeatures(degree=2, include_bias=False)

# 创建多项式特征
polynomial_interation.fit_transform(features)

array([[2., 3., 4., 6., 9.],
       [2., 3., 4., 6., 9.],
       [2., 3., 4., 6., 9.]])

### 4.5 转换特征
#### 问题描述
对一个或多个特征进行自定义的转换(应用函数)。
#### 解决方案
使用 scikit-learn 的 FunctionTransformer() 方法对一组特征应用一个函数。<br>
或者在pandas中可以使用apply进行同样的转换。

In [9]:
import numpy as np
from sklearn.preprocessing import FunctionTransformer

In [10]:
# 创建特征矩阵
features = np.array([[2, 3], [2, 3], [2, 3]])

# 自定义函数
def add_ten(x):
    return x+10

# 创建转换器
ten_transformer = FunctionTransformer(add_ten)

# 转换特征矩阵
ten_transformer.transform(features)

array([[12, 13],
       [12, 13],
       [12, 13]])

---
## <center>异常值</center>

### 4.6 识别异常值
#### 问题描述
识别样本中的一些极端观察值(异常值)。
#### 解决方案
方法一：常用的方法是假设数据是服从正态分布的，基于这个假设，对数据进行分类。服从分布的观察值视为正常值(标注为1)；不服从分布的观察值视为异常值(标注为-1)；

In [11]:
import numpy as np
from sklearn.covariance import EllipticEnvelope
from sklearn.datasets import make_blobs

In [12]:
# 创建模拟数据
features, _ = make_blobs(n_samples=10, n_features=2, centers=1, random_state=1)
features

array([[-1.83198811,  3.52863145],
       [-2.76017908,  5.55121358],
       [-1.61734616,  4.98930508],
       [-0.52579046,  3.3065986 ],
       [ 0.08525186,  3.64528297],
       [-0.79415228,  2.10495117],
       [-1.34052081,  4.15711949],
       [-1.98197711,  4.02243551],
       [-2.18773166,  3.33352125],
       [-0.19745197,  2.34634916]])

In [13]:
# 将第一个观察值的值替换为极端值
features[0,0] = 10000

# 创建识别器
outlier_detector = EllipticEnvelope(contamination=0.1)

# 拟合识别器
outlier_detector.fit(features)

# 预测异常值
outlier_detector.predict(features)

array([-1,  1,  1,  1,  1,  1,  1,  1,  1,  1])

**<font color=blue>make_blobs的用法</font>**

sklearn中的make_blobs函数主要是为了生成数据集的。


make_blobs函数的返回值、参数
```
data, label = make_blobs(n_features=2, n_samples=100, centers=3, random_state=3, cluster_std=[0.8, 2, 5])
```

- n_features表示每一个样本有多少特征值
- n_samples表示样本的个数
- centers是聚类中心点的个数，可以理解为label的种类数
- random_state是随机种子，可以固定生成的数据
- cluster_std设置每个类别的方差

#### 解决方案
方法二：除了查看所有观察值，还可以通过只查看某些特征，并使用**四分位差**(interqutile range，IQR),来识别这些特征的极端值。

In [15]:
# 创建模拟数据
features, _ = make_blobs(n_samples=10, n_features=2, centers=1, random_state=1)

# 创建一个特征
feature = features[:, 0]

# 创建一个函数来返回异常值的下标
def indicies_of_outliners(x):
    q1, q3 = np.percentile(x, [25, 75])
    # IQR是第3个四分位数与第1个四分位数之差
    iqr = q3 - q1
    lower_bound = q1 - (iqr*1.5)
    upper_bound = q1 + (iqr*1.5)
    # 异常值被定义为比第1个四分位数小1.5IQR，或者比第5个四分位数大1.5IQR的值
    return np.where((x>lower_bound) | (x<upper_bound))

indicies_of_outliners(feature)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64),)

**识别异常值并没有通用的解决方案！**

每一种技术都有其优点和缺点，做好的策略是**尝试多种技术**，并从整体上来看结果。

### 4.7 处理异常值
#### 问题描述
处理数据中的异常值。
#### 解决方案
通常有三种方式来处理异常值。<br>
第一种：丢弃它们；<br>第二种：将它们标记为异常值，并作为数据的一个特征；<br>第三种：对有异常值的特征进行转换，从而降低异常值的影响。

In [17]:
import pandas as pd

# 创建数据帧
houses = pd.DataFrame()
houses['Price'] = [534433, 392333, 293222, 4322032]
houses['Bathrooms'] = [2, 3.5, 2, 116]
houses['Square_Feet'] = [1500, 2500, 1500, 48000]

houses

,Price,Bathrooms,Square_Feet
0,534433,2.0,1500
1,392333,3.5,2500
2,293222,2.0,1500
3,4322032,116.0,48000


不难发现，第四条数据与其他三条数据存在极大的差异，属于异常值。
#### 方法一：丢弃

In [18]:
# 筛选符合条件的观察值
houses[houses['Bathrooms']<20]

,Price,Bathrooms,Square_Feet
0,534433,2.0,1500
1,392333,3.5,2500
2,293222,2.0,1500


#### 方法二：进行标记

In [20]:
import numpy as np

# 基于布尔条件语句创建特征
houses['Outlier'] = np.where(houses['Bathrooms']<20, 0, 1)
houses

,Price,Bathrooms,Square_Feet,Outlier
0,534433,2.0,1500,0
1,392333,3.5,2500,0
2,293222,2.0,1500,0
3,4322032,116.0,48000,1


#### 方法三：进行转换

In [22]:
# 对特征取对数值
houses['Log_Of_Square_Feet'] = [np.log(x) for x in houses['Square_Feet']]
houses

,Price,Bathrooms,Square_Feet,Outlier,Log_Of_Square_Feet
0,534433,2.0,1500,0,7.313220
1,392333,3.5,2500,0,7.824046
2,293222,2.0,1500,0,7.313220
3,4322032,116.0,48000,1,10.778956


#### 如何处理异常值
**<font color=blue>如何处理异常值，并不存在一个绝对准则。</font>**<br>
对于异常值到底要如何处理呢？首先，想一想它们为什么是异常值，然后对于数据要有一个最终的目标。<br>

应该基于两个方面来考虑异常值的处理。<br>
第一，要弄清楚是什么原因让它们成为了异常值。<br>
如果你认为它们是错误的观察值，比如它们来着一个错误的采集器或者被计算错误二产生的，那么就要丢弃它们。因为我们<font color=red>无法信任</font>这些数据。<br>
如果你认为这些异常值真的就是极端值(极端情况是存在的)，那么把它们标记为异常值或者对它们的值进行替换，将会是更合理的做法。


第二，基于机器学习的目标来处理异常值。<br>
例如在上述的数据中，如果你需要基于房屋的特征来进行价格的预测，那么假设用100多间卧室的超级豪华住宅是一个不能忽略的数据。<br>
如果你需要使用数据来训练一个用于发布住房信息的模型，那么需要删除异常值。因为在模型中需要假设用户中不存在这样一个想买100多间卧室的豪宅的亿万富翁。

---
## <center>对数据进行操作</center>

### 4.8 将特征离散化
#### 问题描述
将一个数值型特征离散化，分到多个离散的小区间中。
#### 解决方案
根据数据离散化的方式，有两种方法。<br>
第一种方法：根据阈值将特征二值化(二分类)；<br>
第二种方法：根据多个阈值将数值型特征离散化(多分类)。

#### 二分类

In [23]:
import numpy as np
from sklearn.preprocessing import Binarizer

In [24]:
# 创建特征
age = np.array([[6], [12], [20], [36], [65]])

# 创建二值化器
binarizer = Binarizer(18)
# 转换特征
binarizer.fit_transform(age)

array([[0],
       [0],
       [1],
       [1],
       [1]])

#### 多分类

In [25]:
# 将特征离散化
# bins参数中的区间为左闭右开
np.digitize(age, bins=[20, 30, 64])

array([[0],
       [0],
       [1],
       [2],
       [3]], dtype=int64)

### 4.9 使用聚类的方式将观察值分组
#### 问题描述
对观察值进行聚类操作，使相似的观察值被分到一组。
#### 解决方案
使用K-Means(K 均值)聚类法将相似的观察值分到一组，并输出一个新的特征，以标识观察值是属于哪一组。